In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree

import requests
from lxml import html


import re
import hashlib
import sqlite3
import telegram_send
import telebot
import time
import configparser

In [2]:
#Ограничивает длину слов
WORLD_LIMIT = 300

In [3]:


# Создание объекта конфигурации
config = configparser.ConfigParser()

# Чтение данных из файла
config.read('config.ini')

BOT_TOKEN = config.get('tokens', 'BOT_TOKEN')
CHAT_ID = config.get('tokens', 'CHAT_ID')

## Функции

In [4]:
def get_div(tree, xpath_format, div_mode=True):
    list_divs = []


    # Выбор элемента
    selected_element = tree.xpath(xpath_format)


    for div in selected_element:
        # Получение HTML-разметки только для выбранного элемента
        html_representation = html.tostring(div, encoding='unicode', method='html')

        list_divs.append(html_representation)
    return list_divs


# Parse

In [5]:
dict_info = {}



d_mus_names = \
{
'https://victorymuseum.ru/playbill/' :'Музей победы',
'https://blokadamus.ru/музей-обороны-и-блокады-ленинграда/события/':'Музей блокады Ленинграда',
'http://www.cmaf.ru/':'Музей минобороны в Москве',
'https://1418museum.ru/news/':'Музей дорогами победы (парк Патриот)',
'https://1812panorama.ru/afisha':'Бородинская битва',
'https://sovrhistory.ru/exhibitions/future/index.php':'Музей современной истории России',
'https://moscowmanege.ru/ru/vystavki-all/':'Выставочный центр Манеж, Москва',
'https://mosmuseum.ru/exhibitions/':'Музей Москвы',
'https://mediashm.ru/?all=1' :'ГИМ, Москва',
'https://www.museikino.ru/media_center/':'Музей кино, Москва',
'https://ugvim.ru/news/':"Музей военной истории «Свердловск: Говорит Москва!» ",
'https://www.leninmemorial.ru/news/':'Дом-музей В.И. Ленина'
}



list_xpaths = [
'//div[@class="oblique-items"]//div[@class="item"]', 
'//*[@class="elementor-post__card"]', 
'//*[@class="reg-news"]//p[@class="news-item"]/following-sibling::*[1]',
'//*[@class="newsItem__contains"]',
'//*[@class="field-more-title"]',
'//*[@class="exhibitions__text"]',
'//*[@class="news-block exhibition"]'
'//*[@class="articles-grid"]//*[@class="content"]',
'//a[@class="element"]',
'//div[@class="media__inf"]',
'//a[@class="front_cont h_more_w"]',
'//div[@class="row row-news-page"]'


    

              
              ]




for site, xpath  in zip(d_mus_names.keys(), list_xpaths):
    response = requests.get(site,  verify=False)

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        
        
        target_obj = get_div(tree, xpath)  
        dict_info[site] = target_obj
        
        
    else:
        print(f'Ошибка статуса {response.status_code} для {site}')

C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'victorymuseum.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'blokadamus.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '1418museum.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\shums\anaconda3\Lib\site-packages\urllib3\connection

In [6]:
df_raw_p = pd.DataFrame([dict_info]).T

In [7]:
df_raw_p = df_raw_p.explode(0)
df_raw_p = df_raw_p.reset_index()

In [8]:
df_raw_p = df_raw_p.rename(columns={'index':'site', 0:'raw_str'})

In [9]:
df_raw_p[df_raw_p['raw_str'].isna()]

,site,raw_str
45,https://moscowmanege.ru/ru/vystavki-all/,NaN
46,https://mosmuseum.ru/exhibitions/,NaN
47,https://mediashm.ru/?all=1,NaN
48,https://www.museikino.ru/media_center/,NaN
49,https://ugvim.ru/news/,NaN


In [10]:
df_raw_p = df_raw_p[df_raw_p['raw_str'].notna()].copy()

In [11]:
df_raw_p['text_elem'] = df_raw_p['raw_str'].apply(lambda x: BeautifulSoup(x, 'html.parser').text)

In [12]:
df_raw_p = df_raw_p.drop_duplicates(['site', 'text_elem']).copy()
df_raw_p = df_raw_p.reset_index(drop=True).copy()

## Делаем колоку с хэшом

In [13]:
def generate_hash(input_string):
    # Создаем объект хэша SHA-256
    hash_object = hashlib.sha256()

    # Преобразуем входную строку в байты и обновляем хэш
    hash_object.update(input_string.encode('utf-8'))

    # Получаем шестнадцатеричное представление хэша
    hash_value = hash_object.hexdigest()

    return hash_value

In [14]:
# Колонка для хеширования
df_raw_p.insert(0,'hash_col', df_raw_p['site']+ ' ' + df_raw_p['text_elem'])

df_raw_p['hash_col'] = df_raw_p['hash_col'].apply(generate_hash)

In [15]:
df_raw_p.insert(0,'date_stamp', pd.to_datetime('today'))

In [16]:
df_raw_p

,date_stamp,hash_col,site,raw_str,text_elem
0,2023-12-21 22:42:16.198652,fe8f31882e9c6c2b2e8302ff0d650815429526a0ead195...,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_263085"" da...",\n\n\n\n\n\nСобытия\n\nРежим работы в новогодн...
1,2023-12-21 22:42:16.198652,e7ba6078957c72ff37af8ffdf7cd34801b1c40006a7fcb...,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_240023"" da...",\n\n\n\n\n\nСобытия\n\nНовогодний квест «Ёлка ...
2,2023-12-21 22:42:16.198652,5479855f81c1cd0554893695c7322f77b4b4d987b40097...,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_262941"" da...",\n\n\n\n\n\nВыставки\n\nНовый год. Тыл приближ...
3,2023-12-21 22:42:16.198652,211782c6057dc2827995bf94de910f6f1ec453f4b75d6f...,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_244446"" da...",\n\n\n\n\n\nСобытия\n\nОлимпиада «Музеи. Парки...
4,2023-12-21 22:42:16.198652,168f242498124c5f4d824cc9432f3e95aaff7f2618a263...,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_262872"" da...",\n\n\n\n\n\nСобытия\n\n«Битва за Москву. Перва...
5,2023-12-21 22:42:16.198652,c79a66aa8bdf2fd9a83b302b661a25ad5314a67d896171...,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_181829"" da...",\n\n\n\n\n\nВыставки\n\nZаветам Vерны\nВыставк...
6,2023-12-21 22:42:16.198652,48a698dde1142727169f4a36eadab25a5d054adecbebc4...,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_263109"" da...",\n\n\n\n\n\nСобытия\n\nПроекты Музея приняли у...
7,2023-12-21 22:42:16.198652,a46f348ab4f78ef4d5c2ae4442f66cba397f08fc2eea0c...,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_263098"" da...","\n\n\n\n\n\nВыставки\n\nБез права на славу, во..."
8,2023-12-21 22:42:16.198652,160f358f603f607a05115a8b580f24375505486a34975b...,https://victorymuseum.ru/playbill/,"<div class=""item"" id=""bx_3218110189_263001"" da...",\n\n\n\n\n\nСобытия\n\nЗмей Горыныч на горе\nП...
9,2023-12-21 22:42:16.198652,46f431e37c46439bfbb6dfc70264f0314ca9d86c6cb687...,https://blokadamus.ru/музей-обороны-и-блокады-...,"<div class=""elementor-post__card"">\n\t\t\t\t<a...",\n\n\n\n\n\t\t\t\tВНИМАНИЕ! Анонс! 26 декабря ...


## Загружаем в db

In [17]:
conn = sqlite3.connect('museums.db')

In [18]:
# Создаю таблицы

qu = '''
    CREATE TABLE if not EXISTS "parse" (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE,
    "site" TEXT,
    "raw_str" TEXT,
    "text_elem" TEXT
)
'''
conn.execute(qu)

qu = '''
    CREATE TABLE if not EXISTS "parse_temp" (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE,
    "site" TEXT,
    "raw_str" TEXT,
    "text_elem" TEXT

)
'''
conn.execute(qu)

# Добавляю таблицу, для успешных выполнений
qu = '''
    CREATE TABLE if not EXISTS send_telegram (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE, 
    teleg_mes TEXT
)
'''
conn.execute(qu)

In [19]:
# Временная таблица
df_raw_p.to_sql('parse_temp', conn, index=False, if_exists='replace')


# Импортирую значения с новыми значениями по хэшу
qu = '''
INSERT INTO parse
select *
from parse_temp
where hash_col not in (select hash_col from parse)

'''
conn.execute(qu)

In [20]:
qu = '''
select *
from parse
where hash_col not in (select hash_col from send_telegram)
order by date_stamp
'''
df_n_compl = pd.read_sql(qu, conn)

In [21]:
df_n_compl.shape

(0, 5)

# Чистка

In [22]:
def adj_lim(x:list):
    '''Ограничивает длину слова'''
    for i, el  in enumerate(x):

        if len(el) > WORLD_LIMIT:
            x[i] = el[:WORLD_LIMIT]+'...'
            
    return x



list_unuse = ['\t\t\t\tЧитать далее»\t\t\t', 'подробнее >>']


def drop_words(x:list):
    '''
    Удаляю лишние слова 
    '''
    new_list = []
    for l in x:

        if not l in list_unuse:
            new_list.append(l)
    return new_list

        

In [23]:
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('^[(\\n)\\t ]+','', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('(\\n){3,}','\n\n', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace(' {3,}',' ', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('\\n+$', '', regex=True)

In [24]:
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(lambda x: re.split(r"(\n)", x))

df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(adj_lim)

df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(drop_words)

In [25]:
df_n_compl['text_elem']= df_n_compl['text_elem'].apply(lambda x: ''.join(x))

In [26]:
df_n_compl

,date_stamp,hash_col,site,raw_str,text_elem


# Telegram

## Добавляем имена музеев

In [27]:
df_n_compl['mus_name'] = df_n_compl['site'].map(d_mus_names)

df_n_compl['mus_name']= df_n_compl['mus_name'].fillna(df_n_compl['site'])


## Отправка

### Подготовка

In [28]:
df_n_compl['teleg_mes'] = '[' + df_n_compl.mus_name + ']' + '(' + df_n_compl.site + ')' + '\n'

df_n_compl['teleg_mes'] = df_n_compl['teleg_mes']+ df_n_compl.text_elem

In [29]:
df_n_compl

,date_stamp,hash_col,site,raw_str,text_elem,mus_name,teleg_mes


In [30]:
df_mes = df_n_compl[['hash_col', 'teleg_mes']].copy()

### Исполнение

In [35]:
# Убирайю символ не разрывного пробела
df_mes['teleg_mes'] = df_mes['teleg_mes'].str.replace('\xa0', ' ', regex=True)

In [36]:
#df_mes.loc[0, 'teleg_mes'] = '[https://blokadamus.ru/музей-обороны-и-блокады-ленинграда/события/](https://blokadamus.ru/музей-обороны-и-блокады-ленинграда/события/)\nХранитель фонда Изобразительного искусства Государственного мемориального музея обороны и блокады Ленинграда Елена Харлашова рассказала об уникальности выставке «Пронзительные краски блокады» — игре цвета\t\t\t\n\n'

In [37]:
df_mes

,hash_col,teleg_mes


In [38]:
# Замените 'YOUR_BOT_TOKEN' на токен вашего бота
bot = telebot.TeleBot(BOT_TOKEN)

# Замените 'YOUR_CHAT_ID' на ID чата, куда вы хотите отправить сообщение
chat_id = CHAT_ID

PAUSE_ITER = 20
PAUSE_TIME = 50


# Ожидание перд запуском
# time.sleep(PAUSE_TIME)
for num, tuple_mes in enumerate(df_mes.itertuples()):
    
    try:
        # Отправка сообщения
        bot.send_message(chat_id, tuple_mes.teleg_mes, parse_mode='Markdown')
    
        # Сохранение успешных сообщений
        df = pd.DataFrame([tuple_mes]).drop(columns='Index')

        df['date_stamp'] = pd.to_datetime('now')

        if num % 19==0 and num!=0:
            print('Ожидание')
            time.sleep(PAUSE_TIME)

        df.to_sql('send_telegram', conn, if_exists='append', index=False)
    except Exception as send_error:
        print(num, send_error, tuple_mes)
        df_error = pd.DataFrame([[tuple_mes.hash_col, num,send_error, 
                                              tuple_mes.teleg_mes]], columns=['hash_col','num', 'error', 'teleg_mes'])

        df_error.insert(0,'date_time', pd.to_datetime('now'))

        df_error.to_sql('send_error', conn, index=False, if_exists='append')

    
    
